<a href="https://colab.research.google.com/github/MarceloGarcia92/1.PersonalProject_HandModel/blob/main/Improving_Model_Crop_Images_CO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from numpy import load
from numpy import genfromtxt

In [ ]:
Dimension = '3D'
#Dimension = '2D'

Size = '160x160'
#Size = '240x240'

#Label = 'Bones'
Label = 'Joins'
#Label = 'Fingers'

Model = 'Transfer_Learning'
#Model = 'Created'

In [ ]:
Type_Model = Model + '_' + Dimension
Type_Data = Dimension + '_' + Label

path = '/content/drive/MyDrive/1. Personal Project - Marcelo Garcia/AI_Project/Hand/New_Points/' + Type_Data + '/' 
path_models = '/content/drive/MyDrive/1. Personal Project - Marcelo Garcia/AI_Project/Hand/Models/' + Type_Model + '/' + Label + '/'
path_x = '/content/drive/MyDrive/1. Personal Project - Marcelo Garcia/AI_Project/Hand/New_Points/' + Type_Data + '/' 

In [ ]:
x_train = load(path + 'Train/' + 'x_train_' + Dimension + '_' + Label + '_' + Size + '.npy')
x_validate = load(path + 'Validate/' + 'x_validate_' + Dimension + '_' + Label + '_' + Size + '.npy')
x_test = load(path_x + 'Test/' +  'x_test_' + Dimension + '_' + Label + '_' + Size + '.npy')

In [ ]:
x_train = load('/content/drive/MyDrive/1. Personal Project - Marcelo Garcia/AI_Project/Hand/New_Points/3D_Joins/Train/x_train_3D_Joins_160x160.npy')
x_validate = load('/content/drive/MyDrive/1. Personal Project - Marcelo Garcia/AI_Project/Hand/New_Points/3D_Joins/Validate/x_validate_3D_Joins_160x160.npy')
x_test = load('/content/drive/MyDrive/1. Personal Project - Marcelo Garcia/AI_Project/Hand/New_Points/3D_Joins/Test/x_test_3D_Joins_160x160.npy')

In [ ]:
print(x_train.shape, x_validate.shape, x_test.shape)

(11853, 160, 160, 3) (1482, 160, 160, 3) (1482, 160, 160, 3)


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>=0.95):
            print("\nReached 95.0% accuracy so cancelling training!")
            self.model.stop_training = True

Expected_performance_acc = myCallback()

monitor_val_acc = tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', patience = 4)

In [ ]:
def Improving(points, epochs, history_list, saved_list):
  history_list = []
  saved_list = []
  for point in range(0, len(points)):
  
    Selected_model = tf.keras.models.load_model(path_models  + points[point] + '_model_' + Type_Model + '_' + Size +'.h5', custom_objects = {'KerasLayer': hub.KerasLayer})
    test = genfromtxt(path + 'Test/' + points[point] + '_Test_' + Dimension + '_' + Size + '.csv', delimiter=",")
    train = genfromtxt(path + 'Train/' + points[point] + '_train_' + Dimension + '_' + Size + '.csv', delimiter=",")
    validate = genfromtxt(path + 'Validate/' + points[point] + '_Validate_' + Dimension + '_' + Size + '.csv', delimiter=",")

    [x_1, y_1] = Selected_model.evaluate(x_test, test,verbose=1)

    new_model = Selected_model.fit(x_train, train, epochs=epochs, batch_size= 128, verbose= 1, validation_data=(x_validate, validate)) #callbacks = [monitor_val_acc, Expected_performance_acc]
    [x_2, y_2] = Selected_model.evaluate(x_test, test,verbose=1)

    if y_2 > y_1:
      Selected_model.save(path_models  + points[point] + '_model_' + Type_Model + '_' + Size +'.h5')
      history_list.append(points[point] + '_model save')
      saved_list.append(points[point])
      print(points[point] + '_model save', ', First Test Accuracy: '+ "{:.0%}".format(y_1), ', Second Test Accuracy: '+ "{:.0%}".format(y_2), ', Improved Accuracy: '+ "{:.0%}".format(y_2 - y_1))
      
    else:
      history_list.append(points[point] + '_model not save')
      print(points[point] + '_model not save', ', First Test Accuracy: '+ "{:.0%}".format(y_1), ', Second Test Accuracy: '+ "{:.0%}".format(y_2), ', Lost Accuracy: '+ "{:.0%}".format(y_2 - y_1))

  print(history_list)

In [ ]:
list_joins = ['cero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen', 'eighteen', 'nineteen', 'twenty']
list_bones = ['Carpals_1', 'Metacarpals_1', 'Proxima_phalanges_1', 'Distal_phalanges_1', 'Metacarpals_2', 'Proxima_phalanges_2', 'Intermediate_phalanges_2', 'Distal_phalanges_2', 'Metacarpals_3', 'Proxima_phalanges_3', 'Intermediate_phalanges_3', 'Distal_phalanges_3', 'Metacarpals_4', 'Proxima_phalanges_4', 'Intermediate_phalanges_4', 'Distal_phalanges_4', 'Metacarpals_5', 'Proxima_phalanges_5', 'Intermediate_phalanges_5', 'Distal_phalanges_5']
list_fingers = ['Thumb', 'Index_finger', 'Middle_finger', 'Ring_finger', 'Pinky']

In [ ]:
Improving(list_joins, 50, 'history', 'saved')

47/47 [==============================] - 4s 68ms/step - loss: 0.0074 - accuracy: 0.9136
Epoch 1/50
93/93 [==============================] - 82s 877ms/step - loss: 0.0021 - accuracy: 0.9672 - val_loss: 0.0066 - val_accuracy: 0.9143
Epoch 2/50
93/93 [==============================] - 82s 879ms/step - loss: 0.0024 - accuracy: 0.9638 - val_loss: 0.0065 - val_accuracy: 0.9170
Epoch 3/50
93/93 [==============================] - 82s 880ms/step - loss: 0.0022 - accuracy: 0.9688 - val_loss: 0.0065 - val_accuracy: 0.9177
Epoch 4/50
93/93 [==============================] - 82s 880ms/step - loss: 0.0022 - accuracy: 0.9674 - val_loss: 0.0068 - val_accuracy: 0.9136
Epoch 5/50
93/93 [==============================] - 82s 880ms/step - loss: 0.0021 - accuracy: 0.9682 - val_loss: 0.0070 - val_accuracy: 0.9123
Epoch 6/50
93/93 [==============================] - 82s 879ms/step - loss: 0.0023 - accuracy: 0.9673 - val_loss: 0.0068 - val_accuracy: 0.9042
Epoch 7/50
93/93 [==============================] - 82